In [1]:
import pandas as pd
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import accuracy_score, auc, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
## 0. 打印设置
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)  ## 显示全部结果，不带省略点
# pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.0f}'.format)

In [2]:
## 1.1 读取数据
train_Base = pd.read_csv(r"data/train.csv")
test_Base = pd.read_csv(r"data/test.csv")

In [3]:
## 1.2 数据合并
data = pd.concat([test_Base, train_Base], axis=0)
data

,policy_id,age,customer_months,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud
0,681822,60,473,2002-12-17,B,500/1000,1000,1135,0,445975,FEMALE,MD,exec-managerial,camping,other-relative,0,-44262,2015-01-31,Multi-vehicle Collision,Rear Collision,Total Loss,Other,S2,Arlington,9,2,?,0,3,?,53253,5212,10251,39503,Saab,95,2006,nan
1,301288,36,173,1994-01-15,B,100/300,1000,916,0,469238,FEMALE,Masters,exec-managerial,camping,other-relative,0,-38591,2015-01-04,Multi-vehicle Collision,Rear Collision,Major Damage,Ambulance,S5,Springfield,22,3,?,0,0,NO,69401,8309,8439,50012,Mercedes,ML350,2008,nan
2,212001,36,147,1995-12-19,B,500/1000,1000,1176,5000000,595953,FEMALE,MD,adm-clerical,hiking,own-child,56753,0,2015-02-09,Single Vehicle Collision,Side Collision,Total Loss,Police,S6,Northbrook,0,1,NO,2,0,NO,63919,5572,11477,42801,Dodge,Neon,2009,nan
3,797680,24,71,1992-06-20,C,500/1000,500,1472,0,613103,FEMALE,High School,armed-forces,yachting,other-relative,0,0,2015-01-15,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,S1,Springfield,23,3,?,0,0,NO,63173,12027,6500,43423,Dodge,RAM,2012,nan
4,789334,39,230,1996-11-28,C,250/500,1000,1159,4000000,581581,FEMALE,Masters,exec-managerial,reading,wife,0,-62877,2015-01-08,Parked Car,?,Minor Damage,Police,S1,Springfield,17,1,YES,0,0,?,8847,904,1786,6138,Accura,RSX,2003,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1008425,37,196,1997-06-29,C,250/500,500,1301,0,474615,MALE,JD,tech-support,video-games,wife,47627,0,2015-01-18,Single Vehicle Collision,Front Collision,Major Damage,Ambulance,S5,Columbus,4,1,?,0,3,NO,61433,10436,11432,39745,Nissan,Pathfinder,2011,1
696,770702,43,229,2001-05-29,A,250/500,500,1435,8000000,444476,MALE,College,machine-op-inspct,golf,husband,0,-32289,2015-01-13,Multi-vehicle Collision,Rear Collision,Major Damage,Ambulance,S1,Arlington,17,3,NO,0,1,?,68623,6798,14557,50606,Volkswagen,Passat,2013,1
697,755099,35,209,2003-01-11,C,100/300,500,1639,0,639608,FEMALE,College,transport-moving,golf,not-in-family,0,-40797,2015-03-05,Multi-vehicle Collision,Rear Collision,Minor Damage,Fire,S2,Riverwood,7,3,NO,2,0,YES,58033,9129,4598,40740,Mercedes,C300,2002,0
698,693804,44,275,2003-07-22,B,500/1000,2000,1042,0,432061,FEMALE,Associate,machine-op-inspct,paintball,other-relative,46822,0,2015-01-09,Multi-vehicle Collision,Rear Collision,Major Damage,Ambulance,S5,Northbend,20,3,?,1,0,NO,35253,7359,3464,24677,Audi,A3,2007,1


In [4]:
## 1.3 数据清洗
## 1.3.1 索引完善
data.index = range(len(data))

In [5]:
## 1.4 数据探索
## 1.4.1 空值数量
data.isnull().sum()

policy_id                        0
age                              0
customer_months                  0
policy_bind_date                 0
policy_state                     0
policy_csl                       0
policy_deductable                0
policy_annual_premium            0
umbrella_limit                   0
insured_zip                      0
insured_sex                      0
insured_education_level          0
insured_occupation               0
insured_hobbies                  0
insured_relationship             0
capital-gains                    0
capital-loss                     0
incident_date                    0
incident_type                    0
collision_type                   0
incident_severity                0
authorities_contacted            0
incident_state                   0
incident_city                    0
incident_hour_of_the_day         0
number_of_vehicles_involved      0
property_damage                  0
bodily_injuries                  0
witnesses           

In [6]:
# 1.4.2 唯一值个数
for col in data.columns:
    print(col, data[col].nunique())

policy_id 1000
age 45
customer_months 385
policy_bind_date 955
policy_state 3
policy_csl 3
policy_deductable 3
policy_annual_premium 996
umbrella_limit 11
insured_zip 999
insured_sex 2
insured_education_level 7
insured_occupation 14
insured_hobbies 20
insured_relationship 6
capital-gains 490
capital-loss 525
incident_date 113
incident_type 4
collision_type 4
incident_severity 4
authorities_contacted 5
incident_state 7
incident_city 7
incident_hour_of_the_day 24
number_of_vehicles_involved 4
property_damage 3
bodily_injuries 3
witnesses 4
police_report_available 3
total_claim_amount 989
injury_claim 945
property_claim 931
vehicle_claim 991
auto_make 14
auto_model 39
auto_year 21
fraud 2


In [7]:
## 1.4.3 字符串的字段，唯一值统计
cat_columns = data.select_dtypes(include='object').columns  

column_name = []
unique_value = []
 
for col in cat_columns:
    column_name.append(col)
    unique_value.append(data[col].nunique())

df = pd.DataFrame()
df['col_name'] = column_name
df['value'] = unique_value
df = df.sort_values('value', ascending=False)
 
df

,col_name,value
0,policy_bind_date,955
8,incident_date,113
18,auto_model,39
6,insured_hobbies,20
5,insured_occupation,14
17,auto_make,14
4,insured_education_level,7
13,incident_state,7
14,incident_city,7
7,insured_relationship,6


In [8]:
## 2 特征工程
## 2.0 特征编码--property_damage、police_report_available
data['property_damage'].value_counts()
data['property_damage'] = data['property_damage'].map({'NO': 0, 'YES': 1, '?': 2})
data['property_damage'].value_counts()

data['police_report_available'].value_counts()
data['police_report_available'] = data['police_report_available'].map({'NO': 0, 'YES': 1, '?': 2})
data['police_report_available'].value_counts()

2    343
0    343
1    314
Name: police_report_available, dtype: int64

In [9]:
## 2.1 特征编码--日期编码
# policy_bind_date, incident_date
data['policy_bind_date'] = pd.to_datetime(data['policy_bind_date'])
data['incident_date'] = pd.to_datetime(data['incident_date'])
 
# 查看最大日期，最小日期
data['policy_bind_date'].min() # 1990-01-08
data['policy_bind_date'].max() # 2015-02-22

data['incident_date'].min() # 2015-01-01
data['incident_date'].max() # 2015-03-01

base_date = data['policy_bind_date'].min()
# 转换为date_diff
data['policy_bind_date_diff'] = (data['policy_bind_date'] - base_date).dt.days
data['incident_date_diff'] = (data['incident_date'] - base_date).dt.days

#去掉原始日期字段 policy_bind_date    incident_date
data.drop(['policy_bind_date', 'incident_date'], axis=1, inplace=True)
data

,policy_id,age,customer_months,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud,policy_bind_date_diff,incident_date_diff
0,681822,60,473,B,500/1000,1000,1135,0,445975,FEMALE,MD,exec-managerial,camping,other-relative,0,-44262,Multi-vehicle Collision,Rear Collision,Total Loss,Other,S2,Arlington,9,2,2,0,3,2,53253,5212,10251,39503,Saab,95,2006,nan,4740,9168
1,301288,36,173,B,100/300,1000,916,0,469238,FEMALE,Masters,exec-managerial,camping,other-relative,0,-38591,Multi-vehicle Collision,Rear Collision,Major Damage,Ambulance,S5,Springfield,22,3,2,0,0,0,69401,8309,8439,50012,Mercedes,ML350,2008,nan,1482,9141
2,212001,36,147,B,500/1000,1000,1176,5000000,595953,FEMALE,MD,adm-clerical,hiking,own-child,56753,0,Single Vehicle Collision,Side Collision,Total Loss,Police,S6,Northbrook,0,1,0,2,0,0,63919,5572,11477,42801,Dodge,Neon,2009,nan,2185,9177
3,797680,24,71,C,500/1000,500,1472,0,613103,FEMALE,High School,armed-forces,yachting,other-relative,0,0,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,S1,Springfield,23,3,2,0,0,0,63173,12027,6500,43423,Dodge,RAM,2012,nan,908,9152
4,789334,39,230,C,250/500,1000,1159,4000000,581581,FEMALE,Masters,exec-managerial,reading,wife,0,-62877,Parked Car,?,Minor Damage,Police,S1,Springfield,17,1,1,0,0,2,8847,904,1786,6138,Accura,RSX,2003,nan,2530,9145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1008425,37,196,C,250/500,500,1301,0,474615,MALE,JD,tech-support,video-games,wife,47627,0,Single Vehicle Collision,Front Collision,Major Damage,Ambulance,S5,Columbus,4,1,2,0,3,0,61433,10436,11432,39745,Nissan,Pathfinder,2011,1,2743,9155
996,770702,43,229,A,250/500,500,1435,8000000,444476,MALE,College,machine-op-inspct,golf,husband,0,-32289,Multi-vehicle Collision,Rear Collision,Major Damage,Ambulance,S1,Arlington,17,3,0,0,1,2,68623,6798,14557,50606,Volkswagen,Passat,2013,1,4173,9150
997,755099,35,209,C,100/300,500,1639,0,639608,FEMALE,College,transport-moving,golf,not-in-family,0,-40797,Multi-vehicle Collision,Rear Collision,Minor Damage,Fire,S2,Riverwood,7,3,0,2,0,1,58033,9129,4598,40740,Mercedes,C300,2002,0,4765,9201
998,693804,44,275,B,500/1000,2000,1042,0,432061,FEMALE,Associate,machine-op-inspct,paintball,other-relative,46822,0,Multi-vehicle Collision,Rear Collision,Major Damage,Ambulance,S5,Northbend,20,3,2,1,0,0,35253,7359,3464,24677,Audi,A3,2007,1,4957,9146


In [10]:
## 2.2 去除无关的特征
data.drop(['policy_id'], axis=1, inplace=True)
data.columns

Index(['age', 'customer_months', 'policy_state', 'policy_csl',
       'policy_deductable', 'policy_annual_premium', 'umbrella_limit',
       'insured_zip', 'insured_sex', 'insured_education_level',
       'insured_occupation', 'insured_hobbies', 'insured_relationship',
       'capital-gains', 'capital-loss', 'incident_type', 'collision_type',
       'incident_severity', 'authorities_contacted', 'incident_state',
       'incident_city', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud', 'policy_bind_date_diff',
       'incident_date_diff'],
      dtype='object')

In [11]:
## 2.3 标签编码
cat_columns = data.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in cat_columns:
    data[col] = le.fit_transform(data[col])
data[cat_columns]

,policy_state,policy_csl,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,auto_make,auto_model
0,1,2,0,4,3,4,2,0,2,2,3,1,0,10,3
1,1,0,0,5,3,4,2,0,2,0,0,4,6,8,24
2,1,2,0,4,0,10,3,2,3,2,4,5,4,4,27
3,2,2,0,2,1,19,2,0,2,1,4,0,6,4,30
4,2,1,0,5,3,15,5,1,0,1,4,0,6,0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,1,1,3,12,18,5,2,1,0,0,4,1,9,29
996,0,1,1,1,6,9,0,0,2,0,0,0,0,13,28
997,2,0,0,1,13,9,1,0,2,1,1,1,5,8,7
998,1,2,0,0,6,13,2,0,2,0,0,4,3,1,4


In [12]:
## 2.4 分箱编码

# ## 1）age分箱
# for x in range(10,70,10):
#     train_Base[train_Base['age'].between(x,x+10)].loc[:,['age']]=x

In [13]:
## 3. 数据集切分
## 3.1 切分训练集和测试集
train = data[data['fraud'].notnull()]
test = data[data['fraud'].isnull()]

In [14]:
## 3.2 训练集中，训练集和验证集的划分

# x_train, x_train_01 = train_test_split(train.drop(['fraud'],axis=1), test_size=0.2, random_state=42)  # 25% of remaining data as validation set  
# y_train, y_train_01 = train_test_split(train['fraud'], test_size=0.2, random_state=42)  # Split labels accordingly  

x_train, x_train_01, y_train, y_train_01 = train_test_split(train.drop(['fraud'],axis=1), train['fraud'], test_size=0.2, random_state=42)

In [15]:
## 4. 模型训练
## 4.1 建立模型
gbm = LGBMClassifier(n_estimators=300, learning_rate=0.01, boosting_type='gbdt',  ## 模型训练超参数 调优参考：https://blog.51cto.com/u_16213313/7201851
                     objective='binary',   ## LGBMClassifier详解： https://blog.csdn.net/yeshang_lady/article/details/118638269
                     max_depth=-1,
                     random_state=2022,
                     metric='auc')

In [16]:
# 4.2 模型训练
## train.drop(['fraud'],axis=1) ## axis=0 表示行，axis=1 表示列
gbm.fit(x_train, y_train)

LGBMClassifier(learning_rate=0.01, metric='auc', n_estimators=300,
               objective='binary', random_state=2022)

In [17]:
# 4.3 模型预测，以proba进行提交，结果会更好
y_train_01_pred = gbm.predict_proba(x_train_01)

In [18]:
## 5. 模型评估
## 5.1 评估auc值
auc = roc_auc_score(y_train_01, y_train_01_pred[:,-1]) # 计算auc值
print("auc值:", auc)
v_code = str(round(auc,5)).split('.')[1]

## 5.2 概率转换
y_train_01_pred[:, 1][y_train_01_pred[:, 1] > 0.5] = '1'
y_train_01_pred[:, 1][y_train_01_pred[:, 1] <= 0.5] = '0'
y_train_01_pred

## 5.3 评估accuracy，precision，recall，f1
from sklearn.metrics import precision_score, recall_score, f1_score
 
accuracy=accuracy_score(y_train_01, y_train_01_pred[:,-1])  ## 计算准确率
precision = precision_score(y_train_01, y_train_01_pred[:,-1]) # 计算精确率
recall = recall_score(y_train_01, y_train_01_pred[:,-1]) # 计算召回率
f1 = f1_score(y_train_01, y_train_01_pred[:,-1]) # 计算F1值


# 输出计算得到的准确率、召回率和F1值
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

auc值: 0.8351020408163264
Accuracy: 0.8285714285714286
Precision: 0.6341463414634146
Recall: 0.7428571428571429
F1 Score: 0.6842105263157895


In [19]:
# ## 5.1 模型命名，版本控制
model_name=f'model_0_{v_code}_base'

In [20]:
## 6 结果输出
## 6.1 test集的预测
y_test_pred = gbm.predict_proba(test.drop(['fraud'],axis=1))
y_test_pred[:, 1][y_test_pred[:, 1] > 0.5] = '1'
y_test_pred[:, 1][y_test_pred[:, 1] <= 0.5] = '0'
y_test_pred

array([[0.92649953, 0.        ],
       [0.42251162, 1.        ],
       [0.95468557, 0.        ],
       [0.88204191, 0.        ],
       [0.92145721, 0.        ],
       [0.94611824, 0.        ],
       [0.93429082, 0.        ],
       [0.95913216, 0.        ],
       [0.95912719, 0.        ],
       [0.87200478, 0.        ],
       [0.96223669, 0.        ],
       [0.92433787, 0.        ],
       [0.4946062 , 1.        ],
       [0.92038303, 0.        ],
       [0.96220802, 0.        ],
       [0.57449847, 0.        ],
       [0.89065875, 0.        ],
       [0.88392134, 0.        ],
       [0.18394689, 1.        ],
       [0.92184911, 0.        ],
       [0.96041894, 0.        ],
       [0.84055293, 0.        ],
       [0.44680206, 1.        ],
       [0.82767871, 0.        ],
       [0.87663028, 0.        ],
       [0.33113612, 1.        ],
       [0.98414679, 0.        ],
       [0.90388546, 0.        ],
       [0.46361061, 1.        ],
       [0.95637928, 0.        ],
       [0.

In [21]:
# 6.2 输出结果

result = pd.read_csv('./data/submission.csv')
result['fraud'] = y_test_pred[:, 1]
result.to_csv(f'./data/{model_name}.csv', index=False)

In [24]:
## 7 模型评估结果输出
evalue_result=pd.read_csv('./data/evalue_result.csv', encoding='utf-8')
evalue_result

,model_name,update_time,Accuracy,Precision,Recall,F1 Score,auc,sub_score,update_content
0,model_0_8496_base,2024/4/26 11:55,1,1,1,1,1,1,base model
1,model_0_8914_base,2024/4/26 13:03,1,1,1,1,1,1,delete# 1.2 1.3 1.4 2.0(date-diff) ; add #3.1(...


In [26]:
## 5.1 模型评估结果输出
import datetime 

new_row = {'model_name': model_name, 'update_time': datetime.datetime.now() , 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall
 , 'F1 Score': f1, 'auc': auc, 'sub_score': 0.8896, 'update_content': 'base on model_0_8496_base; update ## 3.2 -- test_size random_state；update   ## 4.1-- n_estimators'}  
evalue_result.loc[len(evalue_result.index)] = new_row 
evalue_result
evalue_result.to_csv('./data/evalue_result.csv', index=False)